## Visualizing Text with spaCy and other NLP packages

[spaCy](https://spacy.io/) is a free, open-source library for advanced Natural Language Processing (NLP) in Python. It’s designed specifically for production use and helps you build applications that process and “understand” large volumes of text. It can be used to build information extraction or natural language understanding systems.

spaCy is created by [Explosion AI](https://explosion.ai/), which also makes [Prodigy](https://prodi.gy/), which is a scriptable annotation tool for creating training and evaluation data for machine learning.

This tutorial won't have enough time to go through everything about spaCy -- instead, I'll show you a few examples and also provide some helpful visualizer tools that are part of spaCy's universe.

If you're interested in learning more about spaCy, I recommend spaCy's [free online course](https://course.spacy.io/en) as well as its [spaCy 101 documentation](https://spacy.io/usage/spacy-101).

To begin, I'll assume you'll have created a virtual environment and installed the needed packages provided in the [README](README.md).

In [2]:
import spacy

# spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")
text = "Joe Biden is the president of the United States."
doc = nlp(text)

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

/Users/ryan/Downloads/nlp-spacy-visualizations/.env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Joe Biden PERSON
the United States GPE


In [3]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [4]:
from spacy import displacy

displacy.render(doc, style="ent", jupyter=True)

Or we can use it for its dependency parser.

In [5]:
displacy.render(doc, style="dep", jupyter=True)

## Topic modeling with BERTopic

Analyzing new text datasets is challenging because it's hard to know what is the right question to ask. This is where exploratory analysis can help. A popular NLP technique for exploratory analysis is [topic modeling](https://en.wikipedia.org/wiki/Topic_model).

Topic modeling is (typically) is set up as unsupervised machine learning where the goal is to find hidden or latent patterns in datasets that can be interpreated as topics.

We'll use a more modern version of topic modeling using BERTopic.

In [7]:
import pandas as pd

df = pd.read_csv("ready.csv")

df['text'][0]

"dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."

In [8]:
#from sklearn.datasets import fetch_20newsgroups
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP

# Prepare embeddings
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(texts, show_progress_bar=False)

ModuleNotFoundError: No module named 'bertopic'

In [7]:
topic_model.visualize_topics()

ValueError: This BERTopic instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:


# Train BERTopic
topic_model = BERTopic().fit(texts, embeddings)

# Run the visualization with the original embeddings
topic_model.visualize_documents(texts, embeddings=embeddings)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(texts, reduced_embeddings=reduced_embeddings)

## whatlies

A major advance in NLP over the last 10 years has been [word embedding models](https://ruder.io/word-embeddings-1/). These provide a way to quantify word meaning. Embeddings are typically "pre-trained" by learning the context of words that tend to co-occur. A classic example is [word2vec](https://jalammar.github.io/illustrated-word2vec/). One example of this is in [Tensorboard Projector](https://projector.tensorflow.org/).

The [`whatlies`](https://github.com/koaning/whatlies) package tries to help you to understand. "What lies in word embeddings?"

This small library offers tools to make visualisation easier of both word embeddings as well as operations on them. This should be considered an experimental project. This library will allow you to make visualisations of transformations of word embeddings. Some of these transformations are linear algebra operators.

The library uses [`altair`](https://altair-viz.github.io/), which is a Python package for [Vega-Lite](https://vega.github.io/vega-lite/). 

In [10]:
from whatlies import EmbeddingSet
from whatlies.language import SpacyLanguage

lang = SpacyLanguage("en_core_web_sm")
words = ["cat", "dog", "fish", "kitten", "man", "woman",
         "king", "queen", "doctor", "nurse"]

emb = EmbeddingSet(*[lang[w] for w in words])
emb.plot_interactive(x_axis=emb["man"], y_axis=emb["woman"])

/Users/ryan/Downloads/nlp-spacy-visualizations/.env/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

## `bulk`

[`bulk`](https://github.com/koaning/bulk) is a quick developer tool to apply some bulk labels. Given a prepared dataset with 2d embeddings it can generate an interface that allows you to quickly add some bulk, albeit less precice, annotations.

In [11]:
!python -m bulk text ready.csv

About to serve `bulk` over at http://localhost:5006/.
404 GET /favicon.ico (::1) 0.29ms
^C

Aborted!


In [27]:
!python -m bulk text ready.csv --keywords "coffee,pizza,haircuts"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
About to serve `bulk` over at http://localhost:5006/.
^C

Aborted!


If you're interested in learning more, check out Vincent's [Bulk Labeling and Prodigy video](https://www.youtube.com/embed/gDk7_f3ovIk) or the related video on [Bulk Labeling for Images](https://www.youtube.com/watch?v=DmH3JmX3w2I&feature=emb_rel_pause).